# Colin Lefter

## Research question/interests

**What equity data is the most deterministic of the performance of an equity, and of this data, which is the most relevant for a growth portfolio investment strategy such that we can compute an optimized portfolio of equities while using user input to drive our optimization algorithm?**

My research objective is to develop a scalable asset allocation and construction algorithm that implements an objected-oriented design approach. This objective is an outcome of determining what equity data is the most deterministic of the price of an equity, which will be the focus for the majority of the project/

I intend to develop algorithms for constructing multiple linear regressions and Fourier Franforms, among others, that I will then use to construct interactive and statistical models with Plotly and Seaborn. As such, I have a strong interest in the system design of our software and in developing helper functions that can assist all of us with processing data more efficiently. I am also looking forward to using Facebook Prophet[^1] to construct a time series forecast of a sample portfolio recommendation from our software, which can be included in our Tableau Dashboard.

### Analysis Plan
Our objective function is one that takes in a selection of columns from our data sets to then search for the top n companies that satisfy a criteria for having the highest probability of producing an optimal return on investment. These inputs themselves refer to sub-objective functions that take as input user-defined parameters and thresholds that set the criteria for favourable performance attributes. To rank the companies from our data set, and ultimately determine what portion of capital to assign to each equity, I propose a data normalization algorithm that normalizes the data that comprises the favourable subset from each column of our data set. We interpret these normalized values as probabilities of equity selection and ultimately average the score of each company across all columns to then multiply the final score percentage of each company with the total capital specified by the user. In a broad sense, our software is composed of four general classes that include "Data", "Quantitative Analysis", "Data Visualization" and "Portfolio Construction". We inherit the properties from each of these classes to build a functional data analysis chain.

Our data visualization will be concerned with analyzing the influence of certain financial variables, such as Price-to-Earnings, on the price of each equity from a sample of 500 equities (from the S&P 500 index). Such analysis would begin with a statistical summary that will constitute exploratory data analysis, followed by our application of analysis algorithms that we design. The construction of a portfolio is a bonus of our project and will be made possible by the analysis algorithms we have constructed.

**Important Note**
A component of the analysis will involve the comparison of different values of financial variables with the corresponding price of each equity. This constitutes inferential analysis as we are attempting to identify a correlation on the basis of picking stocks based on expected performance. Therefore, this will require us to use past financial data and compare this data with the current price of each equity. As a result, we can only use the 3-month performance data (i.e. 3-month change in share price data) for this comparison as otherwise we would be using future data to predict past performance, which would be invalid.

#### User-defined parameters
Some initial ideas for these parameters include:
- (float) Initial capital
- (float) Additional capital per day, week or month
- (int) Intended holding period (in days)
- (boolean) Importance of dividends (validated based on capital invested)
- (String) Preferred industries (choose from a list, or select all)
- (int) Volatility tolerance (from 0 to 1, 1 indicating that volatility is not important)
- (String) Preferred companies (as a list)[^2]
- (int) Preferred degree of portfolio diversification (from 0 to 1, 1 indicating complete diversification)
- (String) Preferred investment strategy (choose from "Growth", "Value", "GARP")

### Algorithm Plan

####  Tier 1: Threshold-based screening algorithms
- The current plan is to use these algorithms to screen the financial documents from each company by setting a minimum threshold for each financial ratio. This class of algorithms will need to conduct such screening per industry as industry financial ratios are dinstinct from one another.
- A global screening algorithm that selects companies which show favourable performance across all ratios can also be used after each ratio has been individually tested.

#### Tier 2: Regression models
- As of now, the intent is to develop a multiple linear regression model that will attempt to determine a relationship between the yearly and quarterly performance of each company in relation to several columns of data that act as predictors. This can essentially implement the results from the threshold-based screening algorithms to only conduct this analysis on the pre-screened companies.

#### Tier 3: Statistical modelling algorithms
- Tier 3 denotes a class of broadly experimental statistical modelling algorithms that are applied on a pre-final portfolio to add additional points to companies that perform exceptionally well compared to others in the portfolio. For now, these algorithms constitute signal processing algorithms such as a Fourier Transform algorithm that attempts to identify peaks in numerical values that would otherwise not be apparent when examined in isolation and without further processing. Therefore, these algorithms will be used to fine-tune the capital allocation percentages for each company in the pre-final portfolio.

#### Columns of relevance
Data set 1: Overview
- Price
- MKT Cap
- P/E
- EPS
- Sector

Data set 2: Performance
- 1M change (1 month change)
- 3-Month performance
- 6-month perfromance
- YTD performance
- Yearly performance
- Volatility

Data set 3: Valuation
- Price / revenue
- Enterprise value

Data set 4: Dividends
- Dividend yield FWD
- Dividends per share (FY)

Data set 5: Margins
- Gross profit margin
- Operating margin
- Net profit margin

Data set 6: Income Statement
- Gross profit
- Income
- Net cash flow

Data set 7: Balance Sheet
- Current ratio
- Debt/equity
- Quick ratio

The total number of columns would be 24 in this case.

[^1]: This would mean that a few time series data sets would need to be downloaded from TradingView at the end of the project to test the demo porfolio.

[^2]: A helper function can be developed for this, where the user can just type out the name of the company and the ticker is identifed.

In [123]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as mplt
import plotly.graph_objects as go
from IPython.display import display, HTML, Markdown, Latex
import sys

import warnings # remove later
warnings.simplefilter(action='ignore', category=FutureWarning) # remove later

sys.path.append('..')
from analysis.code import project_functions1 as pf

# Plot Configurations

In [124]:
sns.set_theme(style="darkgrid")
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
sns.set(style="ticks", context="talk")
mplt.style.use("dark_background")

config = {
  'toImageButtonOptions': {
    'format': 'png',
    'filename': 'custom_image',
    'height': 3500,
    'width': 2000,
    'scale': 2
  }
}

# Data Loading

In [125]:
equities = pf.EquityData()
performance_df = equities.load_and_process("performance", exclude_columns=['Change 1m, %', 'Change 5m, %', 'Change 15m, %', 'Change 1h, %', 'Change 4h, %', 'Change 1W, %', 'Change 1M, %', 'Change %'])
overview_df = equities.load_and_process("overview", exclude_columns=['Change %', 'Change', 'Technical Rating', 'Volume', 'Volume*Price'], additional_data=performance_df, additional_column='3-Month Performance')
income_statement_df = equities.load_and_process("income_statement", additional_data=performance_df, additional_column='3-Month Performance')
balance_sheet_df = equities.load_and_process("balance_sheet", additional_data=performance_df, additional_column='3-Month Performance')
dividends_df = equities.load_and_process("dividends", exclude_columns=['Price'], additional_data=performance_df, additional_column='3-Month Performance')
margins_df = equities.load_and_process("margins", additional_data=performance_df, additional_column='3-Month Performance')
valuation_df = equities.load_and_process("valuation", exclude_columns=['Price', 'Market Capitalization', 'Price to Earnings Ratio (TTM)', 'Basic EPS (TTM)', 'EPS Diluted (FY)'], additional_data=performance_df, additional_column='3-Month Performance')

### Analysis-Specific Data Wrangling

In [126]:
dfs = [
    overview_df,
    income_statement_df,
    balance_sheet_df,
    dividends_df,
    margins_df,
    performance_df,
    valuation_df
    ]

dfs_names = [
    "Overview Data",
    "Balance Sheet Data",
    "Dividends Data",
    "Income Statement Data",
    "Margins Data",
    "Performance Data",
    "Valuation Data"
    ]

mega_df = equities.combined_data_frame(dfs)
mega_df_no_strings = mega_df.select_dtypes(exclude='object')

mega_df['6-Month Performance'] = performance_df['6-Month Performance']
mega_df['YTD Performance'] = performance_df['YTD Performance']
mega_df['Yearly Performance'] = performance_df['Yearly Performance']

In [127]:
data = pf.EquityData()
quant = pf.QuantitativeAnalysis()
viz = pf.DataVisualization()

# Experimental Feature Development Zone

In [149]:
# weighted values -- procedure:
# using a dictionary
# go through the cor df and for every financial category, log the number of financial vars that have a correlation > 0.7 but < 1 (to avoid duplicates)
# will need to scan both through the rows and columns and take non-nan
# extract this into an array and divide each count by the total number of counts -- each computed value becomes the multiplier for each column
# calculate an aggregated score that will take each column and multiply it by its multiplier and be the average of all such columns for each row
import numpy as np
corr = mega_df.corr(numeric_only=True)

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr_df = corr.mask(mask).dropna(how='all').dropna('columns', how='all')

cols = [col for col in corr_df]
score_count_df = corr_df
score_count_df['Count'] = 0
score_count_df = score_count_df.drop(columns=cols)
score_count_df = score_count_df.T

for col in range(0, len(corr_df.columns)):
    for row in range(0, len(corr_df.index)):
        current_score = abs(corr_df.iloc[row][col])
        if current_score != 1 and current_score >= 0.7:
            df_col_name = corr_df.columns[col]
            df_row_name = corr_df.index[row]
            
            score_count_df[df_col_name] += 1
            score_count_df[df_row_name] += 1

score_count_df

KeyError: 'Price'

In [ ]:
# target_y = 'Market Capitalization Score'
# mega_dfx = mega_df.select_dtypes(exclude='object')
# predictors = [x for x in mega_dfx.columns]
# #predictors = ['Net Income (FY)', 'Number of Employees', 'Current Ratio (MRQ)']

# for predictor in predictors:
#     quant.rank(mega_df, predictor)

# predictors = [x + " Score" for x in predictors]
# predictors
# quant.multiple_linear_regression(mega_df, predictors, target_y) # result for market cap score seems suspicious if you plot the graph

In [ ]:
# fig = px.scatter(
#     mega_df, x=predictors, y=target_y, opacity=0.65,
#     trendline_color_override='darkblue'
# )
# #fig.show()

# Analysis Zone
## Note: to view the interactive graphs plotted, run this analysis notebook in a Jupyter Notebook environment

In [ ]:
# quant.rank(mega_df, 'Price')
# quant.rank(mega_df, 'Market Capitalization')
# quant.rank(mega_df, 'Basic EPS (TTM)')
# mega_df

#quant.multiple_linear_regression(mega_df, ['Basic EPS (TTM) Score'], target_y='Price Score')

In [ ]:
#viz.correlation_plot(mega_df, 'S&P 500 Equity Data')
fig = viz.heatmap_plot(mega_df, 'Correlation Plot of S&P500 Equity Data', number_of_companies=500, correlation_plot=True)

config = {
  'toImageButtonOptions': {
    'format': 'png',
    'filename': 'custom_image',
    'height': 2000,
    'width': 2500,
  }
}

fig.show(config=config)

No correlation:


Values with a correlation coefficient greater than or equal to 0.7 are considered as strong correlations. Likewise, negative correlation coefficients follow the inverse of this criteria. The purpose of this correlation plot is to identify singular variables that are correlated with the positive returns of many other variables that are often considered as benchmarks for strong equity performance. Even more significant is the identification of variables that normally do not have any apparent correlation when viewed in isolation, but do when paired together in a regression. After establishing such occurrences, certain variables can be assigned a stronger weight than others when being processed by the normalization algorithm that is the basis of our equity ranking system used in our asset allocation and construction algorithm. Such variables can later be grouped as predictors into a multiple linear regression model for further analysis. The results from the correlation plot can be classified as follows:

| X-Value(s) | Strong Positive Y-Values (r >= 0.7) |
| --- | --- |
| Market Capitalization | Total Shares Outstanding, Net Income (FY), Gross Profit (FY), Gross Profit (MRQ), EBITDA (TTM), Total Current Assets (MRQ) |
| Basic EPS (TTM) | EPS Diluted (FY), EPS Diluted (TTM), Basic EPS FY |
| EBITDA (TTM), Gross Profit (MRQ), Gross Profit (FY) | Total Shares Outstanding, Enterprise Value (MRQ), Total Current Assets (MRQ), Total Assets (MRQ), Net Income (FY), Gross Profit (FY), Gross Profit (MRQ), Total Debt (MRQ), Last Year Revenue (FY), Total Revenue (FY)|
| Total Revenue (FY) | Total Current Assets (MRQ), Last Year Revenue (FY), Total Assets (MRQ) |
| Last Year Revenue (FY) | Total Current Assets (MRQ), Total Assets (MRQ) |
| Current Ratio (MRQ) | Quick ratio (MRQ), Total Shares Outstanding, Enterprise Value (MRQ) |
| Total Assets (MRQ) | Total Current Assets (MRQ), Total Debt (MRQ) |
| Operating Margin (TTM) | Net Margin (TTM), Pretax Margin (TTM) |
| Enterprise Value (MRQ) | Total Shares Outstanding |
| Number of Employees | Last Year Revenue, Total Revenue (FY) |
| Net Income | Total Shares Outstanding, Enterprise Value, Total Current Assets (MRQ), Total Assets (MRQ) |
| Net Debt | Total Assets (MRQ) |
| Gross Margin (TTM) | Price to Revenue Ratio (TTM), Net Margin (TTM), Pretax Margin (TTM), Operating Margin (TTM) |
| Price to Revenue Ratio (TTM) | Enterprise Value / EBITDA (TTM) |

It should be noted, however, that many of the X values show a high correlation with other Y values due to those Y values being a derivative of the initial X value and vice versa. This observation confirms that  Taking this feature into account can give the following results:

| X-Value(s) | Strong Positive Y-Values (r >= 0.7) |
| --- | --- |
| Market Capitalization | Total Shares Outstanding, Net Income (FY), Gross Profit (FY), Gross Profit (MRQ), EBITDA (TTM), Total Current Assets (MRQ) |
| EBITDA (TTM), Gross Profit (MRQ), Gross Profit (FY) | Total Shares Outstanding, Enterprise Value (MRQ), Total Current Assets (MRQ), Total Assets (MRQ), Net Income (FY), Gross Profit (FY), Gross Profit (MRQ), Total Debt (MRQ), Last Year Revenue (FY), Total Revenue (FY) |
| Current Ratio (MRQ) | Quick ratio (MRQ), Total Shares Outstanding, Enterprise Value (MRQ) |
| Enterprise Value (MRQ) | Total Shares Outstanding |
| Number of Employees | Last Year Revenue, Total Revenue (FY) |
| Net Income | Total Shares Outstanding, Enterprise Value, Total Current Assets (MRQ), Total Assets (MRQ) |
| Price to Revenue Ratio (TTM) | Enterprise Value / EBITDA (TTM) |

Taking variables that may not have any immediate obvious correlation can yield the following:

| X-Value(s) | Strong Positive Y-Values (r >= 0.7) |
| --- | --- |
| EBITDA (TTM), Gross Profit (MRQ), Gross Profit (FY) | Total Shares Outstanding |
| Current Ratio (MRQ) | Total Shares Outstanding |
| Enterprise Value (MRQ) | Total Shares Outstanding |
| Number of Employees | Last Year Revenue, Total Revenue (FY) |
| Net Income | Total Shares Outstanding |

An outcome of these observations can be to take the correlations of these variables and assign them as a multiplier to the normalized values for each respective column to prioritize certain equity data as being more deterministic of a positive return on investment than others.

Another outcome of these observations can be to analyze the density plot creating during the EDA phase and solely focus on the distributions of the above variables from tables 2 or 3 that are skewed towads low normalized values. The logic with this would be that outliers in negatively skewed distributions are more likely to be indicative of stronger performance because they excel in a financial ratio that very few companies excel in.

The weighted scores from each analysis can be aggregated and in the end, a weighted scoring system is used.

**Disclaimer: Only positive values of the correlation coefficients have been used as all negative values exist in the case of dividend-related markers which are out of scope for a growth-portfolio investing strategy, which is the focus of this analysis.**

In [ ]:
config = {
  'toImageButtonOptions': {
    'format': 'png',
    'filename': 'custom_image',
    'height': 2000,
    'width': 2000,
    'scale': 2
  }
}

In [ ]:
predictors = [
    ['Market Capitalization', 'Basic EPS (TTM)', 'EBITDA (TTM)', 'Gross Profit (MRQ)'],
    ['Gross Profit (FY)', 'Total Revenue (FY)', 'Last Year Revenue (FY)', 'Current Ratio (MRQ)'],
    ['Total Assets (MRQ)', 'Operating Margin (TTM)', 'Enterprise Value (MRQ)', 'Number of Employees'],
    ['Net Income (FY)', 'Net Debt (MRQ)', 'Gross Margin (TTM)', 'Price to Revenue Ratio (TTM)']
    ]

In [ ]:
title = 'Faceted Heat Map Grid of Normalized Equity Data from the Top (Left) and Bottom (Right) 20 Companies in the S&P500 Index by Predictor (1 = Best, 0 = Worst)'
display(viz.subplot_generator(mega_df, predictors, title=title, height_reduction_factor=15, width_multiplier=1, vertical_spacing=0.06).show(config=config))

None

The left half of each heatmap subplot illustrates the score distribution of the top 20 companies sorted by the singular independent variable being observed. Conversely, the right half illustrates the bottom 20 companies under the same category. There are 16 independent variables being tested in total, those of which have been selected on the basis of having a high correlation (a positive coefficient of determination that is greater than or equal to 0.7). Some of these independent variables have a high correlation between a wider range of variables than others, which is why the contrast between the number of yellow (indicative of high scores) and blue tiles (indicative of low scores) is greater for those predictors that have a correlation with a greater number of variables.

This binary contrast analysis supports the notion that there are several equity variables that are the most deterministic of the performance of an equity, with performance being interpreted as the ratio of high to low scores for each equity. Therefore, the heatmap facet grid reveals that **Market Capitalization, EBITDA (TTM), Gross Profit (MRQ and FY), Total Revenue (FY), Total Assets (MRQ), Enterprise Value (MRQ) and Net Income (FY)** are the most deterministic of the performance of an equity. Therefore, it is worth prioritizing these financial variables when determining which companies to invest in. However, it should be noted that Enterprise Value (MRQ) includes many of the above variables in its calculation, so it is essentially a derivative of several financial variables.

Taking the top 4 predictors that are the most deterministic of the overall performance of an equity, constructing a binary heat map plot can enable us to better visualize the contrast between the top 20 and bottom 20 companies sorted by predictor.

In [ ]:
predictors = [
    'Market Capitalization',
    'EBITDA (TTM)',
    'Net Income (FY)',
    'Enterprise Value (MRQ)'
    ]

title = 'Binary Faceted Heat Map Grid of the Most Determinstic Equity Data for Overall Performance in the S&P500 Index (1 = Best, 0 = Worst)'
display(viz.binary_subplot_generator(mega_df, predictors, title, 6.2, 1, vertical_spacing=0.03).show(config=config))

None

With **Market Capitalization, EBITDA (TTM), Net Income (FY) and Enterprise Value (MRQ)** being the most deterministic of overall equity performance, the strongest contrast is shown between the top 20 companies and bottom 20 companies.

The last portion of this analysis addresses the **asset allocation** component the project, in which these findings are applied towards the construction of weighted variables to construct an optimized portfolio of equities. The final aspect of the portfolio construction is the use of user input, which prescribes the development of an algorithm that factors user input into the analysis algorithms developed. Therefore, the final product of the analysis uses not only the findings from the statistical algorithms devised, but also the implementation of user input withing these analysis algorithms.

In [ ]:
high_cor_cols = [
    'Market Capitalization',
    'EBITDA (TTM)',
    'Gross Profit (MRQ)',
    'Gross Profit (FY)',
    'Total Revenue (FY)',
    'Total Assets (MRQ)',
    'Enterprise Value (MRQ)',
    'Net Income (FY)'
    ]
low_cor_cols = [
    'Price',
    'Price to Earnings Ratio (TTM)',
    'Free Cash Flow (Annual YoY Growth)',
    'Free Cash Flow (Quarterly YoY Growth)',
    '3-Month Performance',
    'EPS Diluted (FY)',
    'Debt to Equity Ratio (MRQ)',
    'Quick Ratio (MRQ)',
    'Dividend Yield Forward',
    'Dividends per Share (FY)',
    '6-Month Performance', 
    'Yearly Performance'
    ]

mega_df = mega_df[mega_df.columns.drop(list(mega_df.filter(regex='Score')))]

In [ ]:
config = {
  'toImageButtonOptions': {
    'format': 'png',
    'filename': 'custom_image',
    'height': 1100,
    'width': 3300,
    'scale': 2
  }
}

viz.score_density_plot(mega_df, high_cor_cols, 'Density Plots of the Top Predictors of Equity Performance for Companies in the S&P500 Index').show(config=config)
viz.score_density_plot(mega_df, low_cor_cols, 'Density Plot of the Weakest Predictors of Equity Performance for Companies in the S&P500 Index').show(config=config)

The density plot of the top predictors for equity performance exhbits a clear leftwards-skewed distribution trend compared to the density plot of the weakest predictors. These plots further process the excess noise from the faceted binary heat map plot by suggesting that **the top predictors for equity performance are those that most companies perform poorly on**. Therefore, being an outlier in these categories is greatly deterministic of overall performance across all other categories.

In [ ]:
equities = pf.EquityData('processed_us_equities_tradingview_data_')
norm_df = equities.load_and_process('normalized_data', '../data/processed/')
cols = [col + ' Score' for col in high_cor_cols]
#norm_df['Aggregated Score'] = norm_df[cols].mean(axis=1, numeric_only=True)
#norm_df = norm_df[norm_df['Aggregated Score'] > 0.7]
original_norm_df = norm_df

threshold = 0.5
for predictor in high_cor_cols:
    norm_df = norm_df[norm_df[predictor + ' Score'] > threshold]

title = f'Density Plot of the Top Performing (Individual Score > {threshold}) Companies ({len(norm_df)} total) in the S&P500 Index by the Top Predictors for Equity Performance'
viz.score_density_plot(norm_df, cols, title, normalization=False, search_for_score=False).show(config=config)

for predictor in high_cor_cols:
    original_norm_df = original_norm_df[original_norm_df[predictor + ' Score'] < threshold]

title = f'Density Plot of the Worst Performing (Individual Score < {threshold}) Companies ({len(norm_df)} total) in the S&P500 Index by the Top Predictors for Equity Performance'
viz.score_density_plot(original_norm_df, cols, title, normalization=False, search_for_score=False).show(config=config)

The first density plot proves that outliers in these financial variables are indeed the top performering companies as isolating the top performing companies from the entire data set put into the first density plot produces a mirrored image. Consequently, taking the worst performing companies produces an inverse of the first density plot, once again proving that the top predictors for equity performance are those that most companies perform poorly on.